In [1]:
%cd "C:/Users/Patrickn/Jupyter_notebooks/Graduation/AI_Companion/TTS/styletts"

C:\Users\Patrickn\Jupyter_notebooks\Graduation\AI_Companion\TTS\styletts


c:\Users\Patrickn\.conda\envs\tf_pyT\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
!git clone https://huggingface.co/fadi77/pl-bert

Cloning into 'pl-bert'...
Filtering content:  66% (2/3)
Filtering content: 100% (3/3)
Filtering content: 100% (3/3), 814.75 MiB | 1.65 MiB/s
Filtering content: 100% (3/3), 814.75 MiB | 1.38 MiB/s, done.


In [15]:
%ls

 Volume in drive C has no label.
 Volume Serial Number is DAB6-B0D3

 Directory of C:\Users\Patrickn\Jupyter_notebooks\Graduation\AI_Companion\TTS\styletts

05/18/2025  06:18 PM    <DIR>          .
05/18/2025  06:17 PM    <DIR>          ..
05/18/2025  06:18 PM    <DIR>          pl-bert
05/18/2025  06:48 PM             1,906 pl-bert.ipynb
05/18/2025  06:17 PM            36,330 styletts.ipynb
05/18/2025  01:57 PM    <DIR>          StyleTTS2
               2 File(s)         38,236 bytes
               4 Dir(s)  326,338,904,064 bytes free


In [17]:
import os
import yaml
import torch
from transformers import AlbertConfig, AlbertModel

from StyleTTS2.char_indexer import BertCharacterIndexer

symbols = BertCharacterIndexer.symbols

from huggingface_hub import hf_hub_download

class CustomAlbert(AlbertModel):
    def forward(self, *args, **kwargs):
        # Call the original forward method
        outputs = super().forward(*args, **kwargs)

        # Only return the last_hidden_state
        return outputs.last_hidden_state
    
def _load_plbert(config_path, model_path):
    plbert_config = yaml.safe_load(open(config_path))
    
    albert_base_configuration = AlbertConfig(**plbert_config['model_params'], vocab_size=len(symbols))
    bert = CustomAlbert(albert_base_configuration)

    checkpoint = torch.load(model_path, map_location='cpu')
    state_dict = checkpoint['net']
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k.replace('module.', '')
        if name.startswith('encoder.'):
            name = name.replace('encoder.', '')
            new_state_dict[name] = v
    if "embeddings.position_ids" in new_state_dict: del new_state_dict["embeddings.position_ids"]
    bert.load_state_dict(new_state_dict, strict=False)
    
    return bert

def load_plbert(repo_id, dirname):
    config_path = hf_hub_download(
        repo_id=repo_id,
        filename=f"{dirname}/config.yml",
        repo_type="model", 
    )
    
    model_path = hf_hub_download(
        repo_id=repo_id,
        filename=f"{dirname}/model.pth",
        repo_type="model",
    )

    return _load_plbert(config_path, model_path)

bert_model = load_plbert("fadi77/pl-bert", "models/mlm_only_with_diacritics")

C:\Users\Patrickn\AppData\Local\Temp\ipykernel_5312\2601481810.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location='cpu')


In [24]:
print(bert_model)

CustomAlbert(
  (embeddings): AlbertEmbeddings(
    (word_embeddings): Embedding(188, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 128)
    (token_type_embeddings): Embedding(2, 128)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0, inplace=False)
  )
  (encoder): AlbertTransformer(
    (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
    (albert_layer_groups): ModuleList(
      (0): AlbertLayerGroup(
        (albert_layers): ModuleList(
          (0): AlbertLayer(
            (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (attention): AlbertSdpaAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (attention_dropout): Dropout(p=0, inplace=False)
   